# Homework 5
Evelina Teran & Kevin Smith

In [33]:
import numpy as np
import pandas as pd
import platform
import os
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_curve, auc
import matplotlib.pyplot as plt

In [34]:
# Lorenz Loss:
# if(x >1): 0
# else: ln(1+(x-1)**2)

## Question 1

In [35]:
data_direc = os.getcwd() + "/"
if platform.system() == "Windows":
    data_direc = data_direc.replace("/", "\\")
train = pd.read_csv(data_direc + "gisette_train.csv")
train_labels = pd.read_csv(data_direc + "gisette_train_labels.csv")
test = pd.read_csv(data_direc + "gisette_valid.csv")
test_labels = pd.read_csv(data_direc + "gisette_valid_labels.csv")
train = np.delete(train, 5000, axis=1)
test = np.delete(test, 5000, axis=1)


In [53]:
def der_loss(beta, X, y, s):
    n = len(y)
    z = -y* np.dot(X, beta.T)
    return (1/n)* np.dot(-y/(1+np.exp(z)), X) +2*s*beta

s = .0001
mu = 200
iterations = 300


In [54]:
# Assume train and test data are already defined
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

# Define k values
k_values = [10, 30, 100, 300, 500]

# Perform FSA
train_errors, test_errors, train_roc_auc, test_roc_auc, fpr_train, tpr_train, fpr_test, tpr_test = fsa(train, train_labels, test, test_labels, k_values)

# Plot ROC curves
num_features_values = [10, 30, 100, 300, 500]
fig, axs = plt.subplots(2, 3, figsize=(15, 10))

for i, num_features in enumerate(num_features_values):
    ax = axs[i // 3, i % 3]
    ax.plot(fpr_train[i], tpr_train[i], label="Training ROC Curve (AUC = {:.2f})".format(train_roc_auc[i]))
    ax.plot(fpr_test[i], tpr_test[i], label="Test ROC Curve (AUC = {:.2f})".format(test_roc_auc[i]))
    ax.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    ax.set_title(f"ROC Curve for {num_features} features")
    ax.legend(loc="lower right")

plt.tight_layout()
plt.show()

[[-0.78976332 -0.98151706 -0.56563071 ... -0.98656091 -0.85374647
  -0.97413963]
 [-0.97588126 -0.83826867 -0.73350012 ... -0.84535178 -0.99768048
  -0.23458088]
 [-0.3450312   0.         -0.96896925 ... -0.93773312 -0.98355821
  -0.95254402]
 ...
 [-0.97240545  0.          0.         ... -0.3201321  -0.89534685
  -0.97079522]
 [-0.9111512  -0.99695541 -0.9922718  ... -0.93030092 -0.82102374
  -0.99048909]
 [-0.75278298 -0.93179107 -0.78606444 ... -0.93773312 -0.97252579
  -0.29549905]]


ValueError: operands could not be broadcast together with shapes (10,1) (2000,500) (10,1) 

In [52]:
gradient

-1534513.3300920369

## Question 2

In [38]:
train = pd.read_csv(data_direc + "dexter_train.csv", header = None)
train_labels = pd.read_csv(data_direc + "dexter_train_labels.csv", header = None)
test = pd.read_csv(data_direc + "dexter_valid.csv", header = None)
test_labels = pd.read_csv(data_direc + "dexter_valid_labels.csv", header = None)

In [39]:
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

## Question 3

In [40]:
train = pd.read_fwf(data_direc + "madelon_train.data", header = None)
train_labels = pd.read_fwf(data_direc + "madelon_train.labels", header = None)
test = pd.read_fwf(data_direc + "madelon_valid.data", header = None)
test_labels = pd.read_fwf(data_direc + "madelon_valid.labels", header = None)

In [41]:
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)